# Online Retail Store Data Wrangling

## Objective
The objective here is to carry out some necessary data cleaning and prepation actions on the online retail store dataset, transforming the dataset into more usebale form for the several analysis that is intended to be carried out on the dataset, for this project.

In [1]:
#Importing the necessary libraries for analysis.
import pandas as pd
import numpy as np

In [2]:
!pip install requests

In [3]:
path='https://cdn.theforage.com/vinternships/companyassets/ifobHAoMjQs9s6bKS/5XsFFJu2oCLdmYJW2/1654128941410/Online%20Retail.xlsx'

In [4]:
!pip install openpyxl

In [5]:
import requests
import io
resp = requests.get(path).content
df = pd.read_excel(resp)

In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Data Cleaning and Preparation

Checking for some basic informations about the data set.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


### From the information printed, it can be deducted that:
* The data set comprise of 541,909 entries (rows), 0 to 541,908.
* The data set has a total of 8 columns, which have been listed along with their constituent datatypes.
* It can be observed that the data set seem to contain some null values, as the entries for columns 'Description' and 'Customer ID' do not seem to correspond to total entries of the data set.

In [8]:
#In a bid to take care of the null entries.
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [9]:
#Removing null entries.
df.dropna(inplace=True)
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
#Converting the data type for customer ID to type integer.
df['CustomerID'] = df['CustomerID'].round(0).astype(int)

In [11]:
#Obtaining basic statistical summary off the data set.
df.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000000,406829.000000,406829.000000
mean,12.061303,3.460471,15287.690570
std,248.693370,69.315162,1713.600303
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13953.000000
50%,5.000000,1.950000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,38970.000000,18287.000000


It can be observed that the columns Quantity and Unit Price has minimum of large negative number and zero respectively. This would distort the analysis and should be removed from the data set.

In [12]:
#Removing entries with quantity and unit price less than zero.
indexQuantity = df[(df['Quantity'] < 0)].index
indexUnitPrice = df[(df['UnitPrice']) <= 0].index
df.drop(indexQuantity, inplace=True)
df.drop(indexUnitPrice, inplace=True)
df.describe()

,Quantity,UnitPrice,CustomerID
count,397884.000000,397884.000000,397884.000000
mean,12.988238,3.116488,15294.423453
std,179.331775,22.097877,1713.141560
min,1.000000,0.001000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


For easier analysis, I'd rather simplify the Invoice Date column by dropping the time part of the data in the Invoice Date column.

In [13]:
#importing the datetime library
import datetime
datetime.datetime.now() #to confirm a successful import.

datetime.datetime(2022, 12, 2, 14, 35, 28, 238078)

In [14]:
#Dropping the time part of the data in the Invoice Date column, as it is unnecessary in the intended analysis.
df.InvoiceDate = pd.to_datetime(df.InvoiceDate).dt.date
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom


Since demographic consideration is of utmost importance to the CEO and CMO, it is thus important that the column for country be given adequate attention.

In [15]:
#Checking for the values in the country column.
print(df.Country.nunique())
print(df.Country.unique())

37
['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'EIRE' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Finland' 'Austria' 'Greece' 'Singapore' 'Lebanon'
 'United Arab Emirates' 'Israel' 'Saudi Arabia' 'Czech Republic' 'Canada'
 'Unspecified' 'Brazil' 'USA' 'European Community' 'Bahrain' 'Malta' 'RSA']


From the above print, it can be seen that the data set is a pull off from 37 countries.
However, it does appears that some of the Countries have irregular name representation. It would therefore make more sense to replace these countries with irregular names representations with a more generally accepted name representation.
After sufficient review and research on the list of countries in the data set, the following countries would get a generally more accepted name representation.
* EIRE would be replaced with Ireland.
* RSA would be replaced with South Africa.

In [16]:
df.Country.replace(['EIRE', 'RSA'], ['Ireland', 'South Africa'], inplace=True)
print(df.Country.nunique())
print(df.Country.unique())

37
['United Kingdom' 'France' 'Australia' 'Netherlands' 'Germany' 'Norway'
 'Ireland' 'Switzerland' 'Spain' 'Poland' 'Portugal' 'Italy' 'Belgium'
 'Lithuania' 'Japan' 'Iceland' 'Channel Islands' 'Denmark' 'Cyprus'
 'Sweden' 'Finland' 'Austria' 'Greece' 'Singapore' 'Lebanon'
 'United Arab Emirates' 'Israel' 'Saudi Arabia' 'Czech Republic' 'Canada'
 'Unspecified' 'Brazil' 'USA' 'European Community' 'Bahrain' 'Malta'
 'South Africa']


I am interedsted in seeing my data set in a regional and continental base. To achieve this, a new column for region and continent has to created and added to the dataframe.

In [17]:
countrysorted = df.Country.unique()
print(sorted(countrysorted))

['Australia', 'Austria', 'Bahrain', 'Belgium', 'Brazil', 'Canada', 'Channel Islands', 'Cyprus', 'Czech Republic', 'Denmark', 'European Community', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania', 'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Saudi Arabia', 'Singapore', 'South Africa', 'Spain', 'Sweden', 'Switzerland', 'USA', 'United Arab Emirates', 'United Kingdom', 'Unspecified']


At this point, the dataset now feels relatively clean, and ready for exploration.
I'd now be saving the cleaned dataset to a directory.

In [18]:
df.to_excel(r'c:\Users\USER\Desktop\Exploratory Data Analysis for an Online Retail Store\\Cleaned Online Retail Data Set.xlsx', index=False, header=True)